# Processing

In [1]:
# Import to be able to import python package from src
import sys
sys.path.insert(0, '../../src')

In [2]:
import pandas as pd
import ontime as on

---
## Generation of random time series

In [3]:
ts = on.generators.random_walk().generate(start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2022-12-31'))

---
## Apply function on the whole time series

with Lambda function

In [4]:
add_two = on.processors.mapper(lambda x : x + 2)
new_ts = add_two.process(ts)
new_ts - ts

<TimeSeries (DataArray) (time: 365, component: 1, sample: 1)> Size: 3kB
array([[[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

...

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]]])
Coordinates:
  * time       (time) datetime64[ns] 3kB 2022-01-01 2022-01-02 ... 2022-12-31
  * component  (component) object 8B 'random_walk'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

with normal function

In [5]:
def add_2(x):
    return x + 2

add_two = on.processors.mapper(add_2)
new_ts = add_two.process(ts)
new_ts - ts

<TimeSeries (DataArray) (time: 365, component: 1, sample: 1)> Size: 3kB
array([[[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

...

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]],

       [[2.]]])
Coordinates:
  * time       (time) datetime64[ns] 3kB 2022-01-01 2022-01-02 ... 2022-12-31
  * component  (component) object 8B 'random_walk'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

---
## Apply Function on Windows of the Time Series

In [6]:
mean = on.processors.windower({
    'function': 'mean',
    'mode': 'rolling',
    'window': 10
})

In [11]:
new_ts = on.TimeSeries.from_darts(mean.process(ts))

In [12]:
ts.plot()

alt.LayerChart(...)

In [13]:
new_ts.plot()

alt.LayerChart(...)

---
## Split Time Series in defined durations of e.g. day, week, month, year

In [14]:
ts = on.generators.random_walk().generate(start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2022-12-31'))

In [15]:
ts.plot()

alt.LayerChart(...)

Split by month

All offset aliases can be used to make different split length (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

In [16]:
seq = ts.split_by_period('M')

/home/benjy/projects_dev/ontime/src/ontime/core/time_series/time_series.py:57: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  splits_df = [g for n, g in df.groupby(pd.Grouper(freq=period))]


In [17]:
len(seq)

12

Group the splits

In [18]:
ts_g = ts.group_splits(seq)

In [19]:
ts_g

<TimeSeries (DataArray) (time: 365, component: 1, sample: 1)> Size: 3kB
array([[[  0.68180208]],

       [[ -0.66313743]],

       [[ -2.00833009]],

       [[ -2.61882592]],

       [[ -2.52592189]],

       [[ -3.03773304]],

       [[ -5.12862143]],

       [[ -3.23933174]],

       [[ -3.61242573]],

       [[ -3.38090137]],

...

       [[-22.67596749]],

       [[-22.40138059]],

       [[-22.28243816]],

       [[-22.55935878]],

       [[-21.24834984]],

       [[-21.00464561]],

       [[-21.78252264]],

       [[-22.49716263]],

       [[-22.44949084]],

       [[-21.68413141]]])
Coordinates:
  * time       (time) datetime64[ns] 3kB 2022-01-01 2022-01-02 ... 2022-12-31
  * component  (component) object 8B 'random_walk'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

---
## Compute Correlation Through Time

Load some data from the Energy dataset in Darts

In [20]:
from darts.datasets import EnergyDataset
ts = EnergyDataset().load()

Get a few columns and samples

In [21]:
cols = ['generation biomass', 'generation solar', 'generation nuclear']
ts = ts[cols][0:1000]

Compute correlations within a daily window

In [22]:
correlation = on.processors.correlation('1D')

In [23]:
ts_corr = correlation.process(ts)

In [25]:
ts_corr[0:100].plot()

alt.LayerChart(...)